Calling packages 

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from cellpose import io, models


import random


The code flow for masking

In [ ]:

# ========= EDIT THESE =========
MODEL_PATH = r"C:\Users\mudzimtb\Downloads\spectra max\25_08_13_Edu_ToPro_RWL_concentrations_1\25_08_13_Edu_ToPro_RWL_concentrations\Expt1\Plate1\IMAGES\train\models\cpsam_20251031_150451"
IMG_DIR    = Path(r"C:\Users\mudzimtb\Downloads\spectra max\25_08_13_Edu_ToPro_RWL_concentrations_1\25_08_13_Edu_ToPro_RWL_concentrations\Expt1\Plate1\IMAGES\test\New folder")
PATTERN    = "*.tif"     # adjust if needed

USE_GPU    = False
DIAMETER   = None        # None = auto; or set e.g. 14.0 for cells (in pixels)
TARGET_CH  = None        # None for single-channel; or set 0/1/2 if (H,W,C)
# =============================


def to_2d_gray(arr, target_ch=None):
    """
    Return a 2D image for Cellpose from:
      (H,W), (H,W,C), (Z,H,W), or (Z,H,W,C)  [channels-last].
    If your files are channels-first, move axes before calling this.
    """
    a = np.asarray(arr)
    if a.ndim == 2:                      # (H,W)
        return a.astype(np.float32)
    if a.ndim == 3:
        if a.shape[-1] <= 8 and min(a.shape[:2]) > 8:   # (H,W,C)
            ch = 0 if target_ch is None else target_ch
            return a[..., ch].astype(np.float32)
        else:                                          # (Z,H,W)
            return a[0].astype(np.float32)             # take first plane
    if a.ndim == 4:                                    # (Z,H,W,C)
        ch = 0 if target_ch is None else target_ch
        return a[0, ..., ch].astype(np.float32)        # first Z, chosen channel
    raise ValueError(f"Unsupported image shape {a.shape}")


def save_preview_with_outlines(bg2d, masks2d, out_png):
    """Gray background + colored contour(s) around each object."""
    plt.figure(figsize=(6,6))
    plt.imshow(bg2d, cmap="gray")
    plt.axis("off")
    labels = np.unique(masks2d)
    labels = labels[labels != 0]
    for lab in labels:
        color = (random.random(), random.random(), random.random())
        plt.contour((masks2d == lab).astype(float), levels=[0.5], linewidths=1.8, colors=[color])
    out_png = Path(out_png)
    out_png.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(out_png, dpi=200, bbox_inches="tight", pad_inches=0)
    plt.close()


def try_save_ij_roi_zip(masks2d: np.ndarray, out_zip: Path):
    """Optional: save ImageJ ROI Manager .zip (needs 'roifile' and 'scikit-image')."""
    try:
        from roifile import ImagejRoi, rois_to_zip
        from skimage import measure

        rois = []
        labels = np.unique(masks2d)
        labels = labels[labels != 0]
        for lab in labels:
            cs = measure.find_contours((masks2d == lab).astype(float), level=0.5)
            if not cs:
                continue
            contour = max(cs, key=lambda c: c.shape[0])  # longest contour
            # contour is (row, col) -> (y, x)
            y, x = contour[:, 0], contour[:, 1]
            roi = ImagejRoi.frompoints(x, y)  # polygon ROI
            roi.name = f"label_{int(lab)}"
            rois.append(roi)

        if rois:
            rois_to_zip(rois, str(out_zip))
            print(f"    Saved ROI zip: {out_zip.name}")
        else:
            print("    (No ROIs found to export)")
    except Exception as e:
        print(f"    (ROI zip skipped) {e.__class__.__name__}: {e}")


def main():
    files = sorted(IMG_DIR.glob(PATTERN))
    if not files:
        raise FileNotFoundError(f"No files found in {IMG_DIR} matching {PATTERN}")

    print("Loading model:")
    print(" ", MODEL_PATH)
    model = models.CellposeModel(pretrained_model=MODEL_PATH, gpu=USE_GPU)

    for img_path in files:
        print(f"Processing {img_path.name} …")
        raw = io.imread(str(img_path))  # preserves 16-bit

        # If a TIFF is channels-first, you can move axes to channels-last:
        # if raw.ndim == 3 and raw.shape[0] <= 4: raw = np.moveaxis(raw, 0, -1)   # (C,H,W)->(H,W,C)
        # if raw.ndim == 4 and raw.shape[1] <= 4: raw = np.moveaxis(raw, 1, -1)   # (Z,C,H,W)->(Z,H,W,C)

        img2d = to_2d_gray(raw, target_ch=TARGET_CH)

        # Segment this single 2D plane
        masks, *_ = model.eval(
            img2d,
            diameter=DIAMETER,
            channels=None,          # already grayscale
            channel_axis=None,
            normalize=True
        )
        masks = masks.astype(np.uint16)

        # Outputs next to each image
        mask_out = img_path.with_name(img_path.stem + "_mask.tif")
       # prev_out = img_path.with_name(img_path.stem + "_preview.png")
     

        io.imsave(str(mask_out), masks)
        print(f"    Saved mask:    {mask_out.name}")

        # save_preview_with_outlines(img2d, masks, prev_out)
      #  print(f"    Saved preview: {prev_out.name}")


    print("Done.")


if __name__ == "__main__":
    main()
